<a href="https://colab.research.google.com/github/FiafRex/Fake-news-Detection-with-deep-learning/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from wordcloud import WordCloud

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from keras.models import load_model

In [ ]:
fake = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/main/data/Fake.csv')

In [ ]:
fake.head()

In [ ]:
fake.columns

In [ ]:
fake['subject'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x= 'subject',data=fake)

In [ ]:
text= ' '.join(fake['text'].tolist())

In [ ]:
' '.join(['this','is','a','data'])

In [ ]:
wordcloud= WordCloud(width=1920, height= 1080).generate(text)
fig = plt.figure(figsize=(10,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
real=pd.read_csv('https://raw.githubusercontent.com/laxmimerit/fake-real-news-dataset/main/data/True.csv')

In [ ]:
text= ' '.join(real['text'].tolist())

In [ ]:
wordcloud= WordCloud(width=1920, height= 1080).generate(text)
fig = plt.figure(figsize=(10,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
real.sample(5)

In [ ]:
unknown_publishers = []
for index, row in enumerate(real.text.values):
      try:
        record = row.split('-', maxsplit=1)
        record[1]


        assert(len(record[0])<120)
      except:
        unknown_publishers.append(index)


In [ ]:
len(unknown_publishers)

In [ ]:
real.iloc[unknown_publishers].text

In [ ]:
real.iloc[8970]

In [ ]:
real=real.drop(8970, axis=0)

In [ ]:
publisher=[]
tmp_text = []

for index, row in enumerate(real.text.values):
  if index in unknown_publishers:
    tmp_text.append(row)
    publisher.append('Unknown')

  else:
    record = row.split('-', maxsplit=1)
    publisher.append(record[0].strip())
    tmp_text.append(record[1].strip())


In [ ]:
real['publisher']=publisher
real['text'] = tmp_text

In [ ]:
real.head()

In [ ]:
real.shape

In [ ]:
empty_fake_index =[index for index, text in enumerate(fake.text.tolist()) if str(text).strip()==""]

In [ ]:
fake.iloc[empty_fake_index]

In [ ]:
real['text']=real['title']+ " " + real['text']
fake['text']=fake['title']+ " " + fake['text']

In [ ]:
real['text']=real['text'].apply(lambda x: str(x).lower())
fake['text']=fake['text'].apply(lambda x: str(x).lower())

In [ ]:
real['class'] = 1
fake['class'] = 0

In [ ]:
real.columns

In [ ]:
real =real[['text', 'class']]

In [ ]:
fake =fake[['text', 'class']]

In [ ]:
data=real.append(fake, ignore_index= True)

In [ ]:
data.sample(5)

In [ ]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

In [ ]:
import preprocess_kgptalkie as ps

In [ ]:
data ['text'].apply(lambda x: ps.remove_special_chars(x))

In [ ]:
import gensim

In [ ]:
y= data['class'].values

In [ ]:
X= [d.split() for d in data['text'].tolist()]

In [ ]:
type(X[0])

In [ ]:
print(X[0])

In [ ]:
DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=X, size=DIM, window=10, min_count=1)

In [ ]:
len(w2v_model.wv.vocab)

In [ ]:
w2v_model.wv['love']

In [ ]:
w2v_model.wv['india']

In [ ]:
w2v_model.wv.most_similar('gandhi')

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

In [ ]:
X  = tokenizer.texts_to_sequences(X)

In [ ]:
#tokenizer.word_index

In [ ]:
plt.hist([len(x) for x in X], bins = 700)
plt.show()

In [ ]:
nos = np.array([len(x) for x in X])
len(nos[nos>1000])

In [ ]:
maxlen =1000
X = pad_sequences(X, maxlen = maxlen)

In [ ]:
len(X[104])

In [ ]:
vocab_size = len(tokenizer.word_index)+1
vocab = tokenizer.word_index

In [ ]:
def get_weight_matrix(model):
  weight_matrix = np.zeros((vocab_size, DIM))

  for word, i in vocab.items():
    weight_matrix[i] = model.wv[word]

    return weight_matrix

In [ ]:
embedding_vectors = get_weight_matrix(w2v_model)

In [ ]:
embedding_vectors.shape

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=DIM, weights = [embedding_vectors], input_length=maxlen, trainable=True))
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y)

In [ ]:
model.fit(X_train, y_train, validation_split=0.3, epochs=6)

In [ ]:
y_pred = (model.predict(X_test) >=0.5).astype(int)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
x=['this is authentic news']

In [ ]:
X_test

In [ ]:
x=['This is authentic news']
x=tokenizer.texts_to_sequences(x)
x= pad_sequences(x,maxlen=maxlen)

In [ ]:
(model.predict(x) >=0.5).astype(int)

In [ ]:
x=["Twitter has told employees that the company's office buildings will be temporarily closed, effective immediately."]
x=tokenizer.texts_to_sequences(x)
x= pad_sequences(x,maxlen=maxlen)
(model.predict(x) >=0.5).astype(int)